In [52]:
import pandas as pd
import geopandas as gpd

In [53]:
# importar capas de ríos
gdf_riachuelo = gpd.read_file("../1_Datos/ACUMAR/RED_HIDRICA_CMR.shp")
gdf_limpolitico = gpd.read_file("../1_Datos/ACUMAR/LIMITE_POLITICO.shp")
gdf_zonas = gpd.read_file("../1_Datos/UNSAM/Zonificación_Localidades_L y LZ.geojson")

# Zona de riachuelo

In [54]:
gdf_riachuelo = gdf_riachuelo.loc[gdf_riachuelo.OBSERVACIO == 'RECTIFICACION']
gdf_riachuelo_buffer = gdf_riachuelo.to_crs("EPSG:5347")
gdf_riachuelo_buffer.geometry = gdf_riachuelo_buffer.buffer(800,   )
gdf_riachuelo_buffer = gdf_riachuelo_buffer.to_crs("EPSG:4326")
gdf_riachuelo_buffer = gdf_riachuelo_buffer.dissolve('OBSERVACIO')

In [55]:
# intersect by limpolitico
gdf_limpolitico = gdf_limpolitico.loc[(gdf_limpolitico.DISTRITO == 'LANUS') | (gdf_limpolitico.DISTRITO == 'LOMAS DE ZAMORA')]
gdf_zona_riachuelo = gdf_riachuelo_buffer.overlay(gdf_limpolitico, how='intersection')

In [56]:
gdf_zona_riachuelo.rename(columns={'DISTRITO':'municipio'}, inplace = True)
gdf_zona_riachuelo['id'] = 15
gdf_zona_riachuelo['localidad'] = 'x'
gdf_zona_riachuelo.loc[gdf_zona_riachuelo.municipio == 'LANUS', 'localidad'] = 'SIRGA - Lanús'
gdf_zona_riachuelo.loc[gdf_zona_riachuelo.municipio == 'LANUS', 'municipio'] = 'Lanús'
gdf_zona_riachuelo.loc[gdf_zona_riachuelo.municipio == 'LOMAS DE ZAMORA', 'id'] = 16
gdf_zona_riachuelo.loc[gdf_zona_riachuelo.municipio == 'LOMAS DE ZAMORA', 'localidad'] = 'SIRGA - Lomas de Zamora'
gdf_zona_riachuelo.loc[gdf_zona_riachuelo.municipio == 'LOMAS DE ZAMORA', 'municipio'] = 'Lomas de Zamora'
gdf_zona_riachuelo = gdf_zona_riachuelo[['id', 'municipio', 'localidad', 'geometry']]

# Merge all zonas

In [57]:
gdf_zonas_dif = gdf_zonas.overlay(gdf_riachuelo_buffer, how='difference')

In [58]:
gdf_zonas_complete = pd.concat([gdf_zonas_dif, gdf_zona_riachuelo])
gdf_zonas_complete.reset_index(inplace=True, drop=True)
gdf_zonas_complete

,id,municipio,localidad,geometry
0,1,Lanús,Valentín Alsina,"POLYGON ((-58.39896 -34.66938, -58.39763 -34.6..."
1,2,Lanús,Lanús Oeste,"POLYGON ((-58.42518 -34.67557, -58.42490 -34.6..."
2,6,Lanús,Monte Chingolo,"POLYGON ((-58.38091 -34.73822, -58.38024 -34.7..."
3,5,Lanús,Lanús Este,"POLYGON ((-58.36398 -34.70055, -58.36396 -34.7..."
4,3,Lanús,Gerli,"POLYGON ((-58.36398 -34.70055, -58.36446 -34.7..."
5,4,Lanús,Remedios de Escalada,"POLYGON ((-58.42806 -34.71350, -58.42377 -34.7..."
6,7,Lomas de Zamora,Villa Fiorito,"POLYGON ((-58.44501 -34.72267, -58.44555 -34.7..."
7,8,Lomas de Zamora,Ingeniero Budge,"MULTIPOLYGON (((-58.44501 -34.72267, -58.44372..."
8,12,Lomas de Zamora,Llavallol,"POLYGON ((-58.45770 -34.77849, -58.45691 -34.7..."
9,9,Lomas de Zamora,Villa Centenario,"POLYGON ((-58.42806 -34.71350, -58.42788 -34.7..."


In [59]:
gdf_zonas_complete.to_file("./CapasEditadas/Zonificación_Localidades_LyLZ_SIRGA.geojson", driver="GeoJSON")
